In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [45]:
chrome_driver_path = '/Users/scottheng/Projects/linkedin-job-analysis/chromedriver'

sf_swe_jobs_url = 'https://www.linkedin.com/jobs/software-engineering-jobs-san-francisco-bay-area/?geoId=90000084'

# remove this
linkedin_username = 'USERNAME HERE'
linkedin_password = 'PASSWORD HERE'

In [46]:
# initialize driver
driver = webdriver.Chrome(chrome_driver_path)

# connect to linkedin
driver.get('https://www.linkedin.com')

In [47]:
elementID = driver.find_element_by_id('session_key')
elementID.send_keys(linkedin_username)
elementID = driver.find_element_by_id('session_password')
elementID.send_keys(linkedin_password)

elementID.submit()

In [48]:
driver.get(sf_swe_jobs_url)

In [55]:
# get all jobIds
jobSearchResults = driver.find_elements_by_class_name('jobs-search-results__list-item')
jobIdList = []

for result in jobSearchResults:
    attribute = result.get_attribute('data-occludable-entity-urn')
    attribute = attribute.split(':')
    jobIdList.append(attribute[-1])

In [57]:
# make jobId URLs
linkedin_url_prefix = 'https://www.linkedin.com/jobs/view/'
jobIdUrls = []

for jobId in jobIdList:
    url = linkedin_url_prefix + jobId
    jobIdUrls.append(url)

In [59]:
driver.get(jobIdUrls[0])

In [60]:
driver.close()